In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [86]:
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization

The data for this exercise is available at: https://www.kaggle.com/datamunge/sign-language-mnist/home

Sign up and download to find 2 CSV files: sign_mnist_test.csv and sign_mnist_train.csv -- You will upload both of them using this button before you can continue.


In [2]:
uploaded=files.upload()

Saving sign_mnist_test.csv to sign_mnist_test.csv
Saving sign_mnist_train.csv to sign_mnist_train.csv


In [130]:
def get_data(filename):
  # You will need to write code that will read the file passed
  # into this function. The first line contains the column headers
  # so you should ignore it
  # Each successive line contians 785 comma separated values between 0 and 255
  # The first value is the label
  # The rest are the pixel values for that picture
  # The function will return 2 np.array types. One with all the labels
  # One with all the images
  #
  # Tips: 
  # If you read a full line (as 'row') then row[0] has the label
  # and row[1:785] has the 784 pixel values
  # Take a look at np.array_split to turn the 784 pixels into 28x28
  # You are reading in strings, but need the values to be floats
  # Check out np.array().astype for a conversion
    with open(filename) as training_file:
      # Your code starts here
      df = pd.read_csv(filename)
      labels = df.label
      del df['label']
      from sklearn.preprocessing import LabelBinarizer
      label_binarizer = LabelBinarizer()
      labels = label_binarizer.fit_transform(labels)
      images = df.values
      #Using binarizer is more concise, no need to reshape
      #from sklearn.preprocessing import OneHotEncoder
      #labels = np.array(labels).reshape(-1,1)
      #label_binarizer = OneHotEncoder()
      #labels = label_binarizer.fit_transform(labels)
      # Your code ends here
    return images, labels


training_images, training_labels = get_data('sign_mnist_train.csv')
testing_images, testing_labels = get_data('sign_mnist_test.csv')

# Keep these
print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)

# Their output should be:
# (27455, 28, 28)
# (27455,)
# (7172, 28, 28)
# (7172,)

(27455, 784)
(27455, 24)
(7172, 784)
(7172, 24)


In [131]:
# In this section you will have to add another dimension to the data
# So, for example, if your array is (10000, 28, 28)
# You will need to make it (10000, 28, 28, 1)
# Hint: np.expand_dims

training_images = training_images.reshape(-1,28,28,1)# Your Code Here
testing_images = testing_images.reshape(-1,28,28,1)# Your Code Here

# Create an ImageDataGenerator and do Image Augmentation
train_datagen = ImageDataGenerator(
    # Your Code Here
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest'
    )
train_datagen.fit(training_images)

validation_datagen = ImageDataGenerator(
    # Your Code Here
    rescale = 1./255)

validation_datagen.fit(testing_images)

train_generator = train_datagen.flow(
	training_images,training_labels,  batch_size=128
)

validation_generator = validation_datagen.flow(
	testing_images,testing_labels,
  batch_size=128
)
# Keep These
print(training_images.shape)
print(testing_images.shape)
    
# Their output should be:
# (27455, 28, 28, 1)
# (7172, 28, 28, 1)

(27455, 28, 28, 1)
(7172, 28, 28, 1)


In [ ]:
# Define the model
# Use no more than 2 Conv2D and 2 MaxPooling2D
model = tf.keras.models.Sequential([    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(24, activation='softmax')
    # Your Code Here
])

# Compile Model. 
model.compile(loss = 'categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

# Train the Model
history = model.fit(train_generator, epochs=25, steps_per_epoch=20,validation_data = validation_generator, verbose = 1, validation_steps=3)

model.evaluate(testing_images, testing_labels)
    
# The output from model.evaluate should be close to:
[6.92426086682151, 0.56609035]


Epoch 1/25
215/215 [==============================] - 78s 358ms/step - loss: 3.0909 - accuracy: 0.0695 - val_loss: 2.4122 - val_accuracy: 0.2292
Epoch 2/25
215/215 [==============================] - 75s 350ms/step - loss: 2.5289 - accuracy: 0.2040 - val_loss: 1.7278 - val_accuracy: 0.3851
Epoch 3/25
215/215 [==============================] - 76s 352ms/step - loss: 2.1078 - accuracy: 0.3195 - val_loss: 1.3102 - val_accuracy: 0.5718
Epoch 4/25
215/215 [==============================] - 76s 352ms/step - loss: 1.8242 - accuracy: 0.4033 - val_loss: 1.2469 - val_accuracy: 0.5625
Epoch 5/25
215/215 [==============================] - 77s 357ms/step - loss: 1.6312 - accuracy: 0.4635 - val_loss: 0.9659 - val_accuracy: 0.6454
Epoch 6/25
180/215 [========================>.....] - ETA: 11s - loss: 1.4685 - accuracy: 0.5132

In [ ]:
# Plot the chart for accuracy and loss on both training and validation

import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [105]:
training_labels[training_labels==23] = 27